In [ ]:
import pandas as pd
import numpy as np
import torch
import gc
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# catalysts = pd.read_csv('./drive/MyDrive/Colab Notebooks/catalyst_with_sentiment.csv')
# sec = pd.read_csv('./drive/MyDrive/Colab Notebooks/sec.csv').drop(labels='Unnamed: 0', axis=1)
catalysts = pd.read_csv('./data-src/catalyst_with_sentiment.csv')
sec = pd.read_csv('../data-heavy/sec.csv').drop(labels='Unnamed: 0', axis=1) # path to local very heavy file

In [ ]:
sec.head()

,ticker,year,quarter,filing
0,HGEN,2021,QTR2,.07. Submission of Matters to a Vote of Securi...
1,ABEO,2013,QTR4,.02. Departure of Directors or Certain Officer...
2,EGRX,2017,QTR3,.01 Entry into a Material Definitive Agreement...
3,AXGN,2021,QTR4,.02 Results of Operations and Financial Condit...
4,ENTA,2022,QTR2,0001193125-22-178106.txt : 20220622 8-K 1 d356...


In [ ]:
tix_sec = sec.ticker.unique().tolist()
tix_catalysts = catalysts.ticker.unique().tolist()
tix_match = list(set(tix_catalysts).intersection(tix_sec))

In [ ]:
len(tix_match)

602

In [ ]:
catalysts_filt = catalysts.loc[catalysts.ticker.isin(tix_match)].copy()
sec_filt = sec.loc[sec.ticker.isin(tix_match)].copy()

In [ ]:
from transformers import BertForSequenceClassification, BertTokenizer
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
tokenizer = BertTokenizer.from_pretrained('ProsusAI/finbert')
model = BertForSequenceClassification.from_pretrained('ProsusAI/finbert').to(device)

In [ ]:
def sentiment(text):
  gc.collect()
  if len(text) > 8000:
    text = text[:8000]
  tokens = tokenizer.encode_plus(
    text,
    add_special_tokens=True,
    max_length=512, truncation=True,
    padding="max_length"
  )
  tokens = tokenizer.encode_plus(text,
                                 add_special_tokens=False,
                                 return_tensors='pt')
  input_id_chunks = list(tokens['input_ids'][0].split(510))
  mask_chunks = list(tokens['attention_mask'][0].split(510))
  # get required padding length
  for i in range(len(input_id_chunks)):
    pad_len = 512 - input_id_chunks[i].shape[0]
    # check if tensor length satisfies required chunk size
    if pad_len > 0:
        # if padding length is more than 0, we must add padding
        input_id_chunks[i] = torch.cat([
            input_id_chunks[i], torch.Tensor([0] * pad_len)
        ])
        mask_chunks[i] = torch.cat([
            mask_chunks[i], torch.Tensor([0] * pad_len)
        ])
  input_ids = torch.stack(input_id_chunks).to(device)
  attention_mask = torch.stack(mask_chunks).to(device)

  input_dict = {
      'input_ids': input_ids.long(),
      'attention_mask': attention_mask.int()
  }
  outputs = model(**input_dict)
  probs = torch.nn.functional.softmax(outputs[0].to("cpu"), dim=-1).mean(dim=0)
  weights = torch.Tensor([1, -1, 0])
  compound = torch.matmul(probs, weights)
  return compound.item()

In [ ]:
sec_sentiment = sec_filt.filing.apply(sentiment)

Token indices sequence length is longer than the specified maximum sequence length for this model (521 > 512). Running this sequence through the model will result in indexing errors
